### Segmenting and Clustering Neighbourhoods in Toronto

First we will import requests for getting the page details

In [3]:
import requests
import numpy as np
import pandas as pd
import pip    
def install(package):
   pip.main(['install', package])

install('BeautifulSoup4')


In [4]:
wikipedia_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [5]:
page = requests.get(wikipedia_link)
text_html = page.text
text_html

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of postal codes of Canada: M - Wikipedia</title>\n<script>document.documentElement.className = document.documentElement.className.replace( /(^|\\s)client-nojs(\\s|$)/, "$1client-js$2" );</script>\n<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":861324217,"wgRevisionId":861324217,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat"

In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(text_html, 'html.parser')

In [ ]:
print(soup.prettify())

Finding the table from the xml text from wikipedia page

Finding all the  rows of the particular table 

In [9]:
table = soup.find('table')
table_rows = table.find_all('tr')

In [10]:
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

Creating Dataframe from the rows of table

In [11]:
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
df_need = pd.DataFrame(l, columns=["Postcode", "Borough", "Neighbourhood"])

In [12]:
df_need.head()

,Postcode,Borough,Neighbourhood
0,None,None,None
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


Removing the \n the from the data in Neighbourhood

In [13]:
df_need = df_need.replace(r'\n',' ', regex=True)

In [14]:
df_need.head()

,Postcode,Borough,Neighbourhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


##### Filtering the rows that contains Not assigned in Borough

In [15]:
df_filter = df_need[df_need.Borough.str.contains("Not assigned") == False]

In [17]:
df_filter.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


#### Grouping Dataset based on postcode and combining Neighbourhoods

In [18]:
df_combine = df_filter.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()

In [19]:
df_combine.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge ,Malvern"
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union"
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Finding the shape of the Dataframe

In [20]:
df_combine.shape

(103, 3)

# Creating the dataframe from the csv file for coordinates

In [21]:
import sys
import types
import pip

if not('ibm-cos-sdk' in [package.project_name for package in pip.get_installed_distributions()]):
    !pip install ibm-cos-sdk==2.0.0 -q

import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_5732e331cf70432689702fcf5260be08 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='YnB0rne6pRyAH_swz7ug1rj2ZWvXdUzuBy5YG1lKXnl4',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_5732e331cf70432689702fcf5260be08.get_object(Bucket='mydatascienceproject-donotdelete-pr-h4iavryfvgkeuq',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_coord = pd.read_csv(body)
df_coord.head()

Starting new HTTPS connection (1): s3-api.us-geo.objectstorage.service.networklayer.com


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
df_coord = df_coord.rename(columns={'Postal Code': 'Postcode'})

In [23]:
df_final = pd.merge(df_combine,df_coord, on='Postcode')

In [24]:
df_final.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge ,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Creating Clusters of Toronto

In [25]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/ibm/conda/miniconda3

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2018.8.24  |       ha4d7672_0         136 KB  conda-forge
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    conda-4.5.11               |           py35_0         636 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    openssl-1.0.2p             |       h470a237_0         3.5 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0             conda-forge
    geopy:       

In [26]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_final['Borough'].unique()),
        df_final.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


Finding the Co-ordniates of Toronto CA

In [31]:
address = 'Toronto CA'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto Canada are 43.653963, -79.387207.


Creating the map of Toronto Canada

In [36]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_final['Latitude'], df_final['Longitude'], df_final['Borough'], df_final['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Finding the Clusters of Only Toronto

In [40]:
scar_data = df_final[df_final['Borough'] == 'Scarborough'].reset_index(drop=True)
scar_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge ,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park ,Ionview ,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea ,Golden Mile ,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest ,Cliffside ,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff ,Cliffside West",43.692657,-79.264848


In [42]:
address = 'Scarborough CA'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


Creating map for Scarborough

In [46]:
# create map of Manhattan using latitude and longitude values
map_scar = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scar_data['Latitude'], scar_data['Longitude'], scar_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scar)  
    
map_scar

Defining Foursquare Credentials 

In [47]:
CLIENT_ID = '5LQJZTL5NUJTABHIDGBB0FA5AZ40QRKVQ1MDAKZTEECWN0RU' # your Foursquare ID
CLIENT_SECRET = 'PHQBKICY02TYVE3USGYAT3T5R021BXNS0NXFUP4KA4AX4GNX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5LQJZTL5NUJTABHIDGBB0FA5AZ40QRKVQ1MDAKZTEECWN0RU
CLIENT_SECRET:PHQBKICY02TYVE3USGYAT3T5R021BXNS0NXFUP4KA4AX4GNX


Finding first neighbourhood of Scarborough

In [48]:
scar_data.loc[0, 'Neighbourhood']

'Rouge ,Malvern '

Getting the neighbourhood's latitude and longitude

In [49]:
neighborhood_latitude = scar_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scar_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scar_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge ,Malvern  are 43.806686299999996, -79.19435340000001.


Lets find the top 10 venues near neighbourhood

In [50]:
# type your answer here
LIMIT = 10
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url



'https://api.foursquare.com/v2/venues/explore?&client_id=5LQJZTL5NUJTABHIDGBB0FA5AZ40QRKVQ1MDAKZTEECWN0RU&client_secret=PHQBKICY02TYVE3USGYAT3T5R021BXNS0NXFUP4KA4AX4GNX&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=10'

In [51]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bb4ad676a60714462e6de35'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bb6b9446edc76b0d771311c-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16e941735',
         'name': 'Fast Food Restaurant',
         'pluralName': 'Fast Food Restaurants',
         'primary': True,
         'shortName': 'Fast Food'}],
       'id': '4bb6b9446edc76b0d771311c',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Morningside & Sheppard',
        'distance': 387,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80744841934756,
          'ln

Function to get type of category

In [52]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Finding the venues based on location

In [53]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


Function to find the Near by venues

In [55]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [57]:
scar_venues = getNearbyVenues(names=scar_data['Neighbourhood'],
                                  latitudes=scar_data['Latitude'],
                                   longitudes=scar_data['Longitude'])

Rouge ,Malvern 
Highland Creek ,Rouge Hill ,Port Union 
Guildwood ,Morningside ,West Hill 
Woburn 
Cedarbrae 
Scarborough Village 
East Birchmount Park ,Ionview ,Kennedy Park 
Clairlea ,Golden Mile ,Oakridge 
Cliffcrest ,Cliffside ,Scarborough Village West 
Birch Cliff ,Cliffside West 
Dorset Park ,Scarborough Town Centre ,Wexford Heights 
Maryvale ,Wexford 
Agincourt 
Clarks Corners ,Sullivan ,Tam O'Shanter 
Agincourt North ,L'Amoreaux East ,Milliken ,Steeles East 
L'Amoreaux West ,Steeles West 
Upper Rouge 


In [58]:
print(scar_venues.shape)
scar_venues.head()

(79, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge ,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek ,Rouge Hill ,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek ,Rouge Hill ,Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Guildwood ,Morningside ,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood ,Morningside ,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


checking how many venues were returned for each neighborhood

In [60]:
scar_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North ,L'Amoreaux East ,Milliken ,Steeles East",2,2,2,2,2,2
"Birch Cliff ,Cliffside West",4,4,4,4,4,4
Cedarbrae,7,7,7,7,7,7
"Clairlea ,Golden Mile ,Oakridge",9,9,9,9,9,9
"Clarks Corners ,Sullivan ,Tam O'Shanter",10,10,10,10,10,10
"Cliffcrest ,Cliffside ,Scarborough Village West",2,2,2,2,2,2
"Dorset Park ,Scarborough Town Centre ,Wexford Heights",6,6,6,6,6,6
"East Birchmount Park ,Ionview ,Kennedy Park",4,4,4,4,4,4


## Analyzing each neighbourhood

In [61]:
# one hot encoding
scar_onehot = pd.get_dummies(scar_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scar_onehot['Neighborhood'] = scar_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scar_onehot.columns[-1]] + list(scar_onehot.columns[:-1])
scar_onehot = scar_onehot[fixed_columns]

scar_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,...,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Train Station,Vietnamese Restaurant
0,"Rouge ,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek ,Rouge Hill ,Port Union",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek ,Rouge Hill ,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood ,Morningside ,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood ,Morningside ,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Finding the mean by groupng the same categories

In [62]:
scar_grouped = scar_onehot.groupby('Neighborhood').mean().reset_index()
scar_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,...,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Train Station,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.00,0.000000
1,"Agincourt North ,L'Amoreaux East ,Milliken ,St...",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.5,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000
2,"Birch Cliff ,Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.00,0.000000
3,Cedarbrae,0.0,0.142857,0.000000,0.142857,0.142857,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.142857,0.00,0.000000
4,"Clairlea ,Golden Mile ,Oakridge",0.0,0.000000,0.000000,0.111111,0.000000,0.0,0.000000,0.222222,0.111111,...,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.111111,0.000000,0.00,0.000000
5,"Clarks Corners ,Sullivan ,Tam O'Shanter",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.100000,0.000000,0.100000,0.00,0.000000,0.000000,0.100000,0.00,0.000000
6,"Cliffcrest ,Cliffside ,Scarborough Village West",0.5,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000
7,"Dorset Park ,Scarborough Town Centre ,Wexford ...",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.166667
8,"East Birchmount Park ,Ionview ,Kennedy Park",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.25,0.000000
9,"Guildwood ,Morningside ,West Hill",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.166667,0.000000,0.000000,...,0.0,0.166667,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000


Printing neighhbourhood with other 5 neighbourhoods

In [64]:
num_top_venues = 5

for hood in scar_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scar_grouped[scar_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt ----
            venue  freq
0  Clothing Store  0.25
1          Lounge  0.25
2  Breakfast Spot  0.25
3    Skating Rink  0.25
4       Pet Store  0.00


----Agincourt North ,L'Amoreaux East ,Milliken ,Steeles East ----
                       venue  freq
0                       Park   0.5
1                 Playground   0.5
2        American Restaurant   0.0
3          Korean Restaurant   0.0
4  Latin American Restaurant   0.0


----Birch Cliff ,Cliffside West ----
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2  General Entertainment  0.25
3                   Café  0.25
4    American Restaurant  0.00


----Cedarbrae ----
                venue  freq
0              Bakery  0.14
1                Bank  0.14
2     Thai Restaurant  0.14
3  Athletics & Sports  0.14
4    Hakka Restaurant  0.14


----Clairlea ,Golden Mile ,Oakridge ----
            venue  freq
0        Bus Line  0.22
1    Intersection  0.11
2          Bakery  0.11
3    S

function to return values in descending order

In [65]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

creating the new dataframe and display the top 10 venues for each neighborhood.

In [69]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scar_grouped['Neighborhood']

for ind in np.arange(scar_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scar_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Skating Rink,Lounge,Breakfast Spot,Vietnamese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
1,"Agincourt North ,L'Amoreaux East ,Milliken ,St...",Park,Playground,Caribbean Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store,Convenience Store
2,"Birch Cliff ,Cliffside West",General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Clothing Store,Hakka Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant
3,Cedarbrae,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Fried Chicken Joint,Coffee Shop,Grocery Store,General Entertainment
4,"Clairlea ,Golden Mile ,Oakridge",Bus Line,Intersection,Bus Station,Metro Station,Soccer Field,Ice Cream Shop,Bakery,Park,Convenience Store,Department Store
5,"Clarks Corners ,Sullivan ,Tam O'Shanter",Noodle House,Thai Restaurant,Fried Chicken Joint,Fast Food Restaurant,Italian Restaurant,Shopping Mall,Rental Car Location,Pizza Place,Coffee Shop,Chinese Restaurant
6,"Cliffcrest ,Cliffside ,Scarborough Village West",American Restaurant,Motel,Clothing Store,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
7,"Dorset Park ,Scarborough Town Centre ,Wexford ...",Indian Restaurant,Pet Store,Latin American Restaurant,Vietnamese Restaurant,Chinese Restaurant,Bank,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint
8,"East Birchmount Park ,Ionview ,Kennedy Park",Train Station,Department Store,Convenience Store,Coffee Shop,Clothing Store,Ice Cream Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint
9,"Guildwood ,Morningside ,West Hill",Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Clothing Store,General Entertainment,Fried Chicken Joint


Creating 16 clusters for finding proper data

In [108]:
# set number of clusters
kclusters = 16
scar_grouped_clustering = scar_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scar_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:17] 

array([ 9,  5,  8, 15, 12,  2,  4,  7, 11, 13,  6, 14,  0,  3,  1, 10], dtype=int32)

Creating Labels for each cluster

In [110]:
scar_merged = scar_data

# add clustering labels
scar_merged['Cluster Labels'] = kmeans.labels_.sort

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scar_merged = scar_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

scar_merged.head() # check the last columns!



,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge ,Malvern",43.806686,-79.194353,<built-in method sort of numpy.ndarray object ...,Fast Food Restaurant,Vietnamese Restaurant,Indian Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Department Store,Convenience Store
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union",43.784535,-79.160497,<built-in method sort of numpy.ndarray object ...,Bar,Moving Target,Vietnamese Restaurant,Clothing Store,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill",43.763573,-79.188711,<built-in method sort of numpy.ndarray object ...,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Clothing Store,General Entertainment,Fried Chicken Joint
3,M1G,Scarborough,Woburn,43.770992,-79.216917,<built-in method sort of numpy.ndarray object ...,Coffee Shop,Korean Restaurant,Mexican Restaurant,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,<built-in method sort of numpy.ndarray object ...,Caribbean Restaurant,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Fried Chicken Joint,Coffee Shop,Grocery Store,General Entertainment


Showing the Clusters on map

In [113]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scar_merged['Latitude'], scar_merged['Longitude'], scar_merged['Neighbourhood'], scar_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters